# Этап 1. Получение данных

Изучим данные, предоставленные сервисом для проекта.

## Импорт библиотек

In [2]:
import pandas as pd

Прочитаем файл *music_project.csv* и сохраним его в переменной *df*. 

In [3]:
df = pd.read_csv('/datasets/music_project.csv')

Получение первых 10 строк таблицы.

In [4]:
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

Общая информация о данных таблицы *df*.




In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB
None


Рассмотрим полученную информацию подробнее.

Всего в таблице 7 столбцов, тип данных у каждого столбца - < напишите название типа данных >.

Подробно разберём, какие в *df* столбцы и какую информацию они содержат:

* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Количество значений в столбцах различается. Это говорит о том, что в данных есть <введите определение> значения.



**Выводы**

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. Данные из столбца *genre* позволят узнать самые популярные жанры.

# Этап 2. Предобработка данных

Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

Получаем перечень названий столбцов. Какая наблюдается проблема — кроме тех, что уже были названы ранее?

In [7]:
df.columns   

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы. Проверим результат.



In [8]:
df.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], axis='columns', inplace=True)

In [9]:
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [10]:
print(df.isnull().sum())

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64


Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем,  не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [11]:
df['track_name'] = df['track_name'].fillna('unknown')

In [12]:
df['artist_name'] = df['artist_name'].fillna('unknown')

In [15]:
df.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Удаляем в столбце с жанрами пустые значения; убеждаемся, что их больше не осталось.

In [17]:
df.dropna(subset = ['genre_name'], inplace = True)

In [169]:
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

Необходимо установить наличие дубликатов.  Если найдутся, удаляем, и проверяем, все ли удалились.

In [170]:
df.duplicated().sum()

3755

In [171]:
df = df.drop_duplicates().reset_index(drop=True)

In [172]:
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Сохраняем список уникальных значений столбца с жанрами в переменной *genres_list*. 

Объявим функцию *find_genre()* для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.





In [19]:
genres_list = df['genre_name'].unique()

In [20]:
# <создание функции find_genre()>
# функция принимает как параметр строку с названием искомого жанра
# в теле объявляется переменная-счётчик, ей присваивается значение 0,
# затем цикл for проходит по списку уникальных значений
# если очередной элемент списка равен параметру функции, 
# то значение счётчика увеличивается на 1
# по окончании работы цикла функция возвращает значение счётчика
def find_genre(genre):
    count = 0
    for row in genres_list:
        if(genre == row):
            count = count+1
    return count

Вызов функции *find_genre()* для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — *hiphop*. Поищем другие варианты:

* hip
* hop
* hip-hop


In [21]:
# <вызовом функции find_genre() проверяется наличие варианта 'hip'>
find_genre('hip')

1

In [22]:
# <проверяется наличие варианта 'hop'>
find_genre('hop')

0

In [23]:
# <проверяется наличие варианта 'hip-hop'>
find_genre('hip-hop')

0

Объявим функцию *find_hip_hop()*, которая заменяет  неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.

In [36]:
# <создание функции find_hip_hop()>
# функция принимает как параметры таблицу df и неверное название
# к столбцу 'genre_name' применяется специальный метод, 
# который заменяет второй параметр на строку 'hiphop'
# результат работы равен подсчитанному методом count() числу значений столбца, 
# которые равны второму параметру
# функция возвращает результат
def find_hip_hop(df, wrong):
    df['genre_name'] = df['genre_name'].replace('hip', 'hiphop')
    total = df[df['genre_name'] == wrong]['genre_name'].count()
    return total

In [37]:
# <замена одного неверного варианта на hiphop вызовом функции find_hip_hop()>
find_hip_hop(df, 'hip')

0

Получаем общую информацию о данных. Убеждаемся, что чистка выполнена успешно.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63881 entries, 0 to 65078
Data columns (total 7 columns):
user_id        63881 non-null object
track_name     63881 non-null object
artist_name    63881 non-null object
genre_name     63881 non-null object
city           63881 non-null object
time           63881 non-null object
weekday        63881 non-null object
dtypes: object(7)
memory usage: 3.9+ MB


**Вывод**

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

# Действительно ли музыку в разных городах слушают по-разному?

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных  в эти дни композиций с известным жанром, и сравниваем результаты.

Группируем данные по городу и вызовом метода *count()* подсчитываем композиции, для которых известен жанр.

In [181]:
# <группировка данных таблицы df по столбцу 'city' и подсчёт количества значений столбца 'genre_name'>
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.

In [34]:
# <группировка данных по столбцу 'weekday' и подсчёт количества значений столбца 'genre_name'>
df.groupby('weekday')['genre_name'].count()

weekday
Friday       22774
Monday       22181
Wednesday    18926
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаём функцию *number_tracks()*, которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [38]:
# <создание функции number_tracks()>
# объявляется функция с тремя параметрами: df, day, city
# в переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'weekday' равно параметру day
# и одновременно значение в столбце 'city' равно параметру city
# в переменной track_list_count сохраняется число значений столбца 'genre_name',
# рассчитанное методом count() для таблицы track_list
# функция возвращает значение track_list_count
def number_tracks(df, day, city):
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [39]:
# <список композиций для Москвы в понедельник>
number_tracks(df, 'Monday', 'Moscow')

16299

In [41]:
# <список композиций для Санкт-Петербурга в понедельник>
number_tracks(df, 'Monday', 'Saint-Petersburg')

5882

In [186]:
# <список композиций для Москвы в среду>
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [42]:
# <список композиций для Санкт-Петербурга в среду>
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

7379

In [47]:
# <список композиций для Москвы в пятницу>
number_tracks(df, 'Friday', 'Moscow')


16610

In [189]:
# <список композиций для Санкт-Петербурга в пятницу>
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.


In [45]:
# <таблица с полученными данными>
data = [['Moscow', 15347, 10865, 15680], ['Saint-Petersburg', 5519, 6913, 5802]]
columns = ['city', 'monday', 'wednesday', 'friday']
table = pd.DataFrame(data=data, columns=columns)
print(table)

               city  monday  wednesday  friday
0            Moscow   15347      10865   15680
1  Saint-Petersburg    5519       6913    5802


**Вывод**

Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

# Утро понедельника и вечер пятницы — разная музыка или одна и та же?

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Получим таблицы данных по Москве *moscow_general* и по Санкт-Петербургу *spb_general*.

In [49]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']
print(moscow_general)

        user_id                     track_name              artist_name  \
1      55204538    Delayed Because of Accident         Andreas Rönnberg   
4      E2DC1FAE                    Soul People               Space Echo   
6      4CB90AA5                           True             Roman Messer   
7      F03E1C1F               Feeling This Way          Polina Griffith   
8      8FA1D3BE       И вновь продолжается бой                  unknown   
...         ...                            ...                      ...   
65073  83A474E7  I Worship Only What You Bleed  The Black Dahlia Murder   
65074  729CBB09                        My Name                   McLean   
65076  C5E3A0D5                      Jalopiina                  unknown   
65077  321D0506                  Freight Train            Chas McDevitt   
65078  3A64EF84      Tell Me Sweet Little Lies             Monica Lopez   

         genre_name    city      time    weekday  
1              rock  Moscow  14:07:09     Friday

In [50]:
# <получение таблицы spb_general>
spb_general = df[df['city'] == 'Saint-Petersburg']

Создаём функцию *genre_weekday()*, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [193]:
# объявление функции genre_weekday() с параметрами df, day, time1, time2
# в переменной genre_list сохраняются те строки df, для которых одновременно:
# 1) значение в столбце 'weekday' равно параметру day,
# 2) значение в столбце 'time' больше time1 и
# 3) меньше time2.
# в переменной genre_list_sorted сохраняются в порядке убывания  
# первые 10 значений Series, полученной подсчётом числа значений 'genre_name'
# сгруппированной по столбцу 'genre_name' таблицы genre_list
# функция возвращает значение genre_list_sorted
def genre_weekday(df, day, time1, time2):
    genre_list = df[(df['weekday'] == day) & (df['time'] >= time1) & (df['time'] <= time2)]
    genre_list_sorted = genre_list.groupby('genre_name')['genre_name'].count()
    genre_list_sorted = genre_list.sort_values(by = 'genre_name', ascending = False)
    return genre_list_sorted


Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [194]:
# <вызов функции для утра понедельника в Москве (вместо df таблица moscow_general)>
genre_weekday(moscow_general, 'Monday','07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
27442,D7FB50DA,Drumming Circle,Professor Trance,worldbeat,Moscow,09:30:47,Monday
20384,F0C5DA9A,Hasta Siempre Comandante,Juan M. Chazarreta,world,Moscow,08:10:09,Monday
36691,C3B36125,Brava la cuoca,Roberto Scaglioni,world,Moscow,09:51:28,Monday
54812,F37EC6D1,Penjereden Dash Gelir,Chingiz Sadykhov,world,Moscow,08:29:31,Monday
58660,59CBCCE5,Dyolmano Dyulbero,The Bulgarian Voices,world,Moscow,09:53:19,Monday
...,...,...,...,...,...,...,...
41545,4EEE1839,Now Is The Time Of Emotion,Prince Rama,alternative,Moscow,08:41:49,Monday
48653,FD2E5F67,Needles,The Pack a.d.,alternative,Moscow,09:39:45,Monday
31390,6E597FD5,Ele,Debo Band,africa,Moscow,08:09:38,Monday
56708,E35FBBF1,Shitor,unknown,africa,Moscow,09:44:18,Monday


In [195]:
# <вызов функции для утра понедельника в Петербурге (вместо df таблица spb_general)>
genre_weekday(spb_general, 'Monday','07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
30068,692AEC35,Baiana,Emicida,world,Saint-Petersburg,09:24:38,Monday
42316,6B62E9CE,The Color of The Night,L' Amore Orchestra,world,Saint-Petersburg,08:56:04,Monday
25542,F018BD36,Aicha,Khaled,world,Saint-Petersburg,09:24:54,Monday
25368,888CB0C0,Sichou Zhi Lu,Ethno Music Orchestra,world,Saint-Petersburg,08:49:11,Monday
44109,2DD633B3,Parole parole,unknown,world,Saint-Petersburg,08:50:08,Monday
...,...,...,...,...,...,...,...
10250,39AA16CE,Hold Tight,900 Miles,alternative,Saint-Petersburg,09:15:28,Monday
25039,BD9360CF,Mama Soul,The Super Supers,alternative,Saint-Petersburg,08:23:31,Monday
44455,2BD2469F,48,Sunny Day Real Estate,alternative,Saint-Petersburg,09:11:50,Monday
26651,5F6841A4,Gib mir Kraft,Eisenwut,alternative,Saint-Petersburg,09:18:22,Monday


In [196]:
# <вызов функции для вечера пятницы в Москве>
genre_weekday(moscow_general, 'Friday','17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
43694,917047E8,The Nomad,Niraj Chag Featuring Melissa Baten,world,Moscow,21:58:43,Friday
30922,3DC9EF93,Aprum Em Keznov,Arabo Ispiryan,world,Moscow,20:07:26,Friday
31820,9F87ADDC,Bir Gozal,unknown,world,Moscow,20:14:37,Friday
52603,4C61D12D,Apple Penne,Bharathvaj,world,Moscow,21:32:05,Friday
31514,5366E775,Die Reblaus,Hans Moser,world,Moscow,21:40:41,Friday
...,...,...,...,...,...,...,...
56971,E27AB390,No More Tears,Art Boulevard,alternative,Moscow,20:43:20,Friday
12728,5A8C45BD,Chicago New York,The Aislers Set,alternative,Moscow,21:03:13,Friday
24797,9EEF2060,Samael the Destroyer,Oceano,alternative,Moscow,21:28:50,Friday
26662,B5496034,Mido Yiduma,Adama Yalomba,africa,Moscow,20:50:05,Friday


In [197]:
# <вызов функции для вечера пятницы в Питере>
genre_weekday(spb_general, 'Friday','17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
54324,54A01EAC,Batwanes Beek Ta'a Ninsa,emad batayeh,world,Saint-Petersburg,20:58:18,Friday
29985,47E5088,Что же делать?,Группа «Домбай»,world,Saint-Petersburg,21:15:04,Friday
44531,873D8A1B,Orkestarsko Oro,Maleševski Melos,world,Saint-Petersburg,20:58:55,Friday
18013,1D507F3F,A Cochabamba Me Voy,Daniele Sepe,world,Saint-Petersburg,21:50:57,Friday
48238,F3FB8127,Dari Lolo,Aravod,world,Saint-Petersburg,21:48:19,Friday
...,...,...,...,...,...,...,...
7639,DFBA0314,Back To You,Kolidescopes,alternative,Saint-Petersburg,20:05:53,Friday
27109,D756A23,Rimango qui,NH3,alternative,Saint-Petersburg,20:14:35,Friday
21793,96B95074,All in My Head,Good Shoes,alternative,Saint-Petersburg,21:32:43,Friday
46559,826C2C51,Your Song,Ben Cocks,adult,Saint-Petersburg,20:48:28,Friday


Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр *world*.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр *world*.

**Вывод**

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

# Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.



Сгруппируем таблицу *moscow_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *moscow_genres*.

Просмотрим первые 10 строк этой новой таблицы.

In [52]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre_name', 
# подсчёт числа значений 'genre_name' в этой группировке методом count(), 
# сортировка Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [201]:
# <просмотр первых 10 строк moscow_genres>
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

Сгруппируем таблицу *spb_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *spb_genres*.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.

In [202]:
# <группировка таблицы spb_general, расчёт, сохранение в spb_genres>
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [203]:
# <просмотр первых 10 строк spb_genres>
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

**Вывод**

В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

# Этап 4. Результаты исследования


Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза **подтверждена**, вторая гипотеза **не подтверждена**  и третья **не подтверждена**.